# Lets import some things

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms


# Decide if cuda

In [2]:
device = torch.device('cpu')
if torch.cuda.is_available():
    print('CUDA available')
    device = torch.device("cuda:0")
print(f'Device: {device}')

Device: cpu


# Load dataset

# 